## Xi book

This script creates a dataframe of Chinese President Xi's book organized by section. 
Thus it is more of a one time used script to create data which can then be used for NLP tasks.

In [1]:
import pandas as pd

import bs4
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
import urllib.request

import time
from lxml import etree
# install lxml and prereqs, note - doing this after creation of venv has given me issues, just remake venv

In [2]:
base_url = 'https://www.12371.cn/special/blqs/xjptzglz/'

In [ ]:
# Get's sections and their urls from the main home page
section_name = []
url_list = []

page = urllib.request.Request(base_url, headers={'User-Agent': 'Mozilla/5.0'}) #being a bit lazy here

uClient = uReq(page)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'lxml')

result_list = page_soup.find_all('li', class_= 'zdy')

for result in result_list:
      urls = result.find_all('a') 

      for u in urls:
            #print(u.text)
            section_name.append(u.text)
            #print(u['href'])
            url_list.append(u['href'])

      print("Sections to be collected: " + str(len(section_name)))

In [ ]:
df = pd.DataFrame({'section': section_name, 'url': url_list})
df.head()

In [ ]:
# Get actual content

url_list = []
section_list = []
text_list = []
title_list = []
date_list = []

for index, row in df.iterrows():
    print('Scraping ' + str(index +1) + ' of ' + str(len(section_name)))
    url_list.append(row.url)
    section_list.append(row.section)

    page = urllib.request.Request(row.url, headers={'User-Agent': 'Mozilla/5.0'}) 

    uClient = uReq(page)
    page_html = uClient.read()
    uClient.close()

    page_soup = soup(page_html, 'lxml')

    text = page_soup.find('div', {'class': 'word'})
    text_list.append(text.text)
   
    title = page_soup.find('h1') #('h1', {'class': 'big_title'})
    title_list.append(title.text)

    date = page_soup.find('h2', {'class': 'small_title'})
    date_list.append(date.text)

    time.sleep(5)

print('Done')

In [6]:
df = pd.DataFrame({'section': section_name, 
                  'url': url_list,
                  'title': title_list,
                  'date': date_list,
                  'text': text_list,
                  })

In [ ]:
df.head(5)

In [10]:
df.to_csv('./DATA/xi_book.csv', index=False)